# Padise 2024
## Masinnägemise töötuba


Valige endale huvipakkuv ülesanne, mida lahendada ning probleemide korral guugeldage või küsige juhendajalt või kaasõpilastelt.


Erinevad ülesanded on erineva raskusastmega, kuid kõik jälgivad sama põhimõtet: 

1. ülesande lahendab **MUDEL** 
2. mudeli tööd demonstreerib **veebikaamera liides**

Siin on antud üldine veebikaamera liidese tsükkel ning sinna sisse tuleb tavaliselt panna mudel, mis töötleb pilte ja tagastab tulemuse veebikaamera väljundisse i.e ekraanile :) 

In [2]:
import cv2, sys, numpy, os 


cap = cv2.VideoCapture(0) #alustab veebikaamera

while True:
    ret, frame = cap.read() #loeb sisse kaadri kaamerast
    if not ret: #kontrollib et midagi ei läinud valesti
        break
    ##
    ##
    ##Siin saab teha kaadritöötlust MUDELI 
    ##
    ##
    cv2.imshow('Väljund', frame) #kuvab pildi veebikaamera väljundisse

    # Programm lõpeb, kui vajutame 'q' 
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Lõpetuseks vabastame kaamera
cap.release()
cv2.destroyAllWindows()

### 1. variant : Näotuvastus  
(kood on suures osas pärit siit : https://www.geeksforgeeks.org/face-detection-using-python-and-opencv-with-webcam/)

Järgnev koodilõik on andmetekogumiseks

In [3]:

haar_file = './naotuvastus/haarcascade_frontalface_default.xml' #mudeli asukoht
datasets = './naotuvastus/datasets' #näopiltide asukoht

sub_data = 'Minu Nimi'	# kaust minu näo piltidest (muuda vastavalt)

path = os.path.join(datasets, sub_data) 
if not os.path.isdir(path): 
	os.mkdir(path) 

# pildi suurus
(width, height) = (130, 100)	 

#'0' on veebikaamera tüüpiliselt, 
# kui on probleeme proovi '1'
face_cascade = cv2.CascadeClassifier(haar_file) #laeme mudeli 

cap = cv2.VideoCapture(0)

    # The program loops until it has 30 images of the face. 
count = 1
while count < 30: #to-do1: teha see osa ümber nii, et programm koguks etteantud x arvu pilte 
				  #to-do2: nõua, et iga järgev pilt oleks eelnevast piisavalt erinev (kasutada nt. eukleidilist kaugust vms.)
				  # või tehakse mingi aja tagant

	ret, frame = cap.read()
	if not ret:
		break

	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
	faces = face_cascade.detectMultiScale(gray, 1.3, 4) 
	for (x, y, w, h) in faces: 
		cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2) 
		face = gray[y:y + h, x:x + w] 
		face_resize = cv2.resize(face, (width, height)) 
		cv2.imwrite('% s/% s.png' % (path, count), face_resize) 
	count += 1
	
	cv2.imshow('OpenCV', frame) 
	if cv2.waitKey(1) & 0xFF == ord('q'):	
		break

# Lõpetuseks vabastame kaamera
cap.release()
cv2.destroyAllWindows()


In [6]:

haar_file = './naotuvastus/haarcascade_frontalface_default.xml' #mudeli asukoht
datasets = './naotuvastus/datasets' #näopiltide asukoht

sub_data = 'Minu Nimi'	# kaust minu näo piltidest (muuda vastavalt)

path = os.path.join(datasets, sub_data) 
if not os.path.isdir(path): 
	os.mkdir(path) 

# pildi suurus
(width, height) = (130, 100)	 

#'0' on veebikaamera tüüpiliselt, 
# kui on probleeme proovi '1'
face_cascade = cv2.CascadeClassifier(haar_file) #laeme mudeli 

cap = cv2.VideoCapture(0)

# See koodijupp teeb veebikaameraga ühe pildi ja kasutab seda hiljem teie ära tundmiseks 
count = 1
#to-do1: teha see osa ümber nii, et programm koguks etteantud x arvu pilte 
#to-do2: nõua, et iga järgev pilt oleks eelnevast piisavalt erinev (kasutada nt. eukleidilist kaugust vms.)
# või tehakse mingi aja tagant

ret, frame = cap.read()
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
faces = face_cascade.detectMultiScale(gray, 1.3, 4) 
for (x, y, w, h) in faces: 
	cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2) 
	face = gray[y:y + h, x:x + w] 
	face_resize = cv2.resize(face, (width, height)) 
	cv2.imwrite('% s/% s.png' % (path, count), face_resize) 
	
cv2.imshow('OpenCV', frame) 

# Lõpetuseks vabastame kaamera
cap.release()
cv2.destroyAllWindows()


Teine osa treenib mudeli tehtud fotode põhjal ja tuvastab näo ja kuvab kui enesekindel mudel on

In [7]:

size = 4

# Tuvastame näo
print('Recognizing Face Please Be in sufficient Lights...') 

# Loome piltide järjendi ja vastavate nimede järjendi 
(images, labels, names, id) = ([], [], {}, 0) 
for (subdirs, dirs, files) in os.walk(datasets): 
	for subdir in dirs: 
		names[id] = subdir 
		subjectpath = os.path.join(datasets, subdir) 
		for filename in os.listdir(subjectpath): 
			path = subjectpath + '/' + filename 
			label = id
			images.append(cv2.imread(path, 0)) 
			labels.append(int(label)) 
		id += 1
(width, height) = (130, 100) 

# Create a Numpy array from the two lists above 
(images, labels) = [numpy.array(lis) for lis in [images, labels]] 

# OpenCV treenib kogutud fotode pealt mudeli
# NOTE FOR OpenCV2: remove '.face' 
model = cv2.face.LBPHFaceRecognizer_create() 
model.train(images, labels) 

# Kasutame fisherRecognizer mudelit veebikaameraga 
face_cascade = cv2.CascadeClassifier(haar_file) 

cap = cv2.VideoCapture(0)

while True:
	ret, frame = cap.read()
	if ret == False:
		break
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
	faces = face_cascade.detectMultiScale(gray, 1.3, 5) 
	for (x, y, w, h) in faces: 
		cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2) 
		face = gray[y:y + h, x:x + w] 
		face_resize = cv2.resize(face, (width, height)) 
		# Proovib nägu tuvastada
		prediction = model.predict(face_resize) 
		cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3) 

		if prediction[1]<500: 

			cv2.putText(frame, '% s - %.0f' %
			    (names[prediction[0]], prediction[1]), (x-10, y-10), 
			   cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0)) 
		else: 
			cv2.putText(frame, 'tundmatu',(x-10, y-10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 255, 0)) 

	cv2.imshow('OpenCV', frame) 

	if cv2.waitKey(1) & 0xFF == ord('q'):
		break
cap.release()
cv2.destroyAllWindows()


Recognizing Face Please Be in sufficient Lights...


1. Proovige tuvastajate erinevates tingimustes (pime, hämar, profiil jne.)
2. Proovige lisada ka kaaslaste nägusid ja treenida nende pealt rohkemate nägude tuvastaja
3. Proovige lahendada to-do1 ja to-do2 ning vaadake, kas programm läheb paremaks 


### 2. variant: Veebikaamerast tahvel (ilmselt ei pane sisse kuna seda on raske tükkideks võtta)

1. Lisage värve
2. Lisage kasutajaliides :)
3. Lisage võimalus pilte salvestada

<img src="./tahvel.jpg" alt="drawing" width="300"/>

(kood: https://medium.com/@serurays/interactive-canvas-drawing-with-hand-tracking-using-opencv-and-python-204c1ca31522)

In [13]:
import numpy as np
import os
from Hand_Tracking_Module import HandDetectorMP

folder_path = "Interface_Pics"
img_list = os.listdir(folder_path)

overlay_list = []

pressed_keys = []

for file in img_list:
    img = cv2.imread(f'{folder_path}/{file}')
    img = cv2.resize(img, (1280, 125))
    overlay_list.append(img)

header = overlay_list[2]

draw_color = (0, 0, 0)

xp, yp = 0, 0

brush_thickness = 15

img_canvas = np.zeros((720, 1280, 3), np.uint8)

cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

detector = HandDetectorMP(detection_con=0.85)

while True:
    success, img = cap.read()

    img = cv2.flip(img, 1)

    img = detector.find_hands(img)

    lm_list = detector.find_position(img, draw=False)

    if len(lm_list) != 0:
        x1, y1 = lm_list[8][1:]
        x2, y2 = lm_list[12][1:]

        fingers = detector.fingers_up()
        total_fingers = fingers.count(1)

        if fingers[1] and fingers[2]:
            xp, yp = 0, 0
            cv2.rectangle(img, (x1, y1 - 15), (x2, y2 + 25), draw_color, cv2.FILLED)

            if y1 < 125:
                if 250 < x1 < 350:
                    header = overlay_list[4]
                    draw_color = (0, 0, 255)
                elif 400 < x1 < 550:
                    header = overlay_list[0]
                    draw_color = (255, 0, 0)
                elif 650 < x1 < 850:
                    header = overlay_list[3]
                    draw_color = (0, 255, 0)
                elif 900 < x1 < 1000:
                    header = overlay_list[1]
                    draw_color = (0, 0, 0)

        elif fingers[1]:
            cv2.circle(img, (x1, y1), 15, draw_color, cv2.FILLED)

            if xp == 0 and yp == 0:
                xp, yp = x1, y1

            cv2.line(img, (xp, yp), (x1, y1), draw_color, brush_thickness)
            cv2.line(img_canvas, (xp, yp), (x1, y1), draw_color, brush_thickness)

            xp, yp = x1, y1

    img_gray = cv2.cvtColor(img_canvas, cv2.COLOR_BGR2GRAY)
    _, img_inv = cv2.threshold(img_gray, 50, 255, cv2.THRESH_BINARY_INV)
    img_inv = cv2.cvtColor(img_inv, cv2.COLOR_GRAY2BGR)
    img = cv2.bitwise_and(img, img_inv)
    img = cv2.bitwise_or(img, img_canvas)

    img[0:125, 0:1280] = header

    cv2.imshow("Canvas", img)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('c'):
        img_canvas = np.zeros((720, 1280, 3), np.uint8)
    elif key == ord('+'):
        brush_thickness += 1
    elif key == ord('-'):
        brush_thickness = max(1, brush_thickness - 1)
    elif 48 <= key <= 57:
        pressed_keys.append(key - 48)

        if len(pressed_keys) == 9:
            blue = int(''.join(map(str, pressed_keys[:3])))
            green = int(''.join(map(str, pressed_keys[4:7])))
            red = int(''.join(map(str, pressed_keys[7:10])))

            draw_color = (blue, green, red)

            print(pressed_keys)

            pressed_keys = []

cv2.destroyAllWindows()
cap.release()

### 3. variant: Dinomäng käežestidega

kood:(https://levelup.gitconnected.com/playing-chromes-dinosaur-game-using-opencv-19b3cf9c3636

https://github.com/harshilp24/handdino-opencv)

todo:mix it up with the mediapipe lib

In [8]:
import math
import pyautogui

In [4]:

# Open Camera
capture = cv2.VideoCapture(0)

while capture.isOpened():

    # Capture frames from the camera
    ret, frame = capture.read()

    # Get hand data from the rectangle sub window
    cv2.rectangle(frame, (100, 100), (300, 300), (0, 255, 0), 0)
    crop_image = frame[100:300, 100:300]

    # Apply Gaussian blur
    blur = cv2.GaussianBlur(crop_image, (3, 3), 0)

    # Change color-space from BGR -> HSV
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

    # Create a binary image with where white will be skin colors and rest is black
    mask2 = cv2.inRange(hsv, np.array([2, 0, 0]), np.array([20, 255, 255]))

    # Kernel for morphological transformation
    kernel = np.ones((5, 5))

    # Apply morphological transformations to filter out the background noise
    dilation = cv2.dilate(mask2, kernel, iterations=1)
    erosion = cv2.erode(dilation, kernel, iterations=1)

    # Apply Gaussian Blur and Threshold
    filtered = cv2.GaussianBlur(erosion, (3, 3), 0)
    ret, thresh = cv2.threshold(filtered, 127, 255, 0)
#####
    # Find contours
    #image, contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours, hierachy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    try:
        # Find contour with maximum area
        contour = max(contours, key=lambda x: cv2.contourArea(x))

        # Create bounding rectangle around the contour
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(crop_image, (x, y), (x + w, y + h), (0, 0, 255), 0)

        # Find convex hull
        hull = cv2.convexHull(contour)

        # Draw contour
        drawing = np.zeros(crop_image.shape, np.uint8)
        cv2.drawContours(drawing, [contour], -1, (0, 255, 0), 0)
        cv2.drawContours(drawing, [hull], -1, (0, 0, 255), 0)

        # Fi convexity defects
        hull = cv2.convexHull(contour, returnPoints=False)
        defects = cv2.convexityDefects(contour, hull)

        # Use cosine rule to find angle of the far point from the start and end point i.e. the convex points (the finger
        # tips) for all defects
        count_defects = 0

        for i in range(defects.shape[0]):
            s, e, f, d = defects[i, 0]
            start = tuple(contour[s][0])
            end = tuple(contour[e][0])
            far = tuple(contour[f][0])

            a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
            b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
            c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
            angle = (math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 180) / 3.14

            # if angle >= 90 draw a circle at the far point
            if angle <= 90:
                count_defects += 1
                cv2.circle(crop_image, far, 1, [0, 0, 255], -1)

            cv2.line(crop_image, start, end, [0, 255, 0], 2)

        # Press SPACE if condition is match

        if count_defects >= 4:
                pyautogui.press('space')
                cv2.putText(frame, "JUMP", (115, 80), cv2.FONT_HERSHEY_SIMPLEX, 2, 2, 2)

        #PLAY RACING GAMES (WASD)
        """
        if count_defects == 1:
            pyautogui.press('w')
            cv2.putText(frame, "W", (115, 80), cv2.FONT_HERSHEY_SIMPLEX, 2, 2, 2)
        if count_defects == 2:
            pyautogui.press('s')
            cv2.putText(frame, "S", (115, 80), cv2.FONT_HERSHEY_SIMPLEX, 2, 2, 2)
        if count_defects == 3:
            pyautogui.press('aw')
            cv2.putText(frame, "aw", (115, 80), cv2.FONT_HERSHEY_SIMPLEX, 2, 2, 2)
        if count_defects == 4:
            pyautogui.press('dw')
            cv2.putText(frame, "dw", (115, 80), cv2.FONT_HERSHEY_SIMPLEX, 2, 2, 2)
        if count_defects == 5:
            pyautogui.press('s')
            cv2.putText(frame, "s", (115, 80), cv2.FONT_HERSHEY_SIMPLEX, 2, 2, 2)
        """

    except:
        pass

    # Show required images
    cv2.imshow("Gesture", frame)

    # Close the camera if 'q' is pressed
    if cv2.waitKey(1) == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()

### 4. variant: Koloriseeri (žesti peale)
1. Koloriseerige üks pilt
2. Tehke veebikaamera mustvalgeks ja siis koloriseerige
3. Lisage mustvalge, koloriseeritud, originaalne režiim žesti peale

<img src="./bw-to-color.jpg" alt="drawing" width="500"/>

(kood: https://www.geeksforgeeks.org/black-and-white-image-colorization-with-opencv-and-deep-learning/)

In [7]:
import numpy as np 
import cv2
from cv2 import dnn

#--------Model file paths--------#
proto_file = './colorization/colorization_deploy_v2.prototxt'
model_file = './colorization/colorization_release_v2.caffemodel'
hull_pts = 'colorization/pts_in_hull.npy'
img_path = 'minbw.jpg'
#--------------#--------------#

#--------Reading the model params--------#
net = dnn.readNetFromCaffe(proto_file,model_file)
kernel = np.load(hull_pts)
#-----------------------------------#---------------------#

#-----Reading and preprocessing image--------#
img = cv2.imread(img_path)
scaled = img.astype("float32") / 255.0
lab_img = cv2.cvtColor(scaled, cv2.COLOR_BGR2LAB)
#-----------------------------------#---------------------#

# add the cluster centers as 1x1 convolutions to the model
class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = kernel.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]
#-----------------------------------#---------------------#

# we'll resize the image for the network
resized = cv2.resize(lab_img, (224, 224))
# split the L channel
L = cv2.split(resized)[0]
# mean subtraction
L -= 50
#-----------------------------------#---------------------#

# predicting the ab channels from the input L channel

net.setInput(cv2.dnn.blobFromImage(L))
ab_channel = net.forward()[0, :, :, :].transpose((1, 2, 0))
# resize the predicted 'ab' volume to the same dimensions as our
# input image
ab_channel = cv2.resize(ab_channel, (img.shape[1], img.shape[0]))


# Take the L channel from the image
L = cv2.split(lab_img)[0]
# Join the L channel with predicted ab channel
colorized = np.concatenate((L[:, :, np.newaxis], ab_channel), axis=2)

# Then convert the image from Lab to BGR 
colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
colorized = np.clip(colorized, 0, 1)

# change the image to 0-255 range and convert it from float32 to int
colorized = (255 * colorized).astype("uint8")

# Let's resize the images and show them together
img = cv2.resize(img,(640,640))
colorized = cv2.resize(colorized,(640,640))

result = cv2.hconcat([img,colorized])

cv2.imshow("Grayscale -> Colour", result)

cv2.waitKey(0)


-1

Üks võimalik lahendus esimesele kahele punktile

In [23]:
# Open Camera
capture = cv2.VideoCapture(0)
 # add the cluster centers as 1x1 convolutions to the model

proto_file = './colorization/colorization_deploy_v2.prototxt'
model_file = './colorization/colorization_release_v2.caffemodel'
hull_pts = 'colorization/pts_in_hull.npy'
img_path = 'minbw.jpg'
#--------------#--------------#

#--------Reading the model params--------#
net = dnn.readNetFromCaffe(proto_file,model_file)
kernel = np.load(hull_pts)
#-----------------------------------#---------------------#


class8 = net.getLayerId("class8_ab")
conv8 = net.getLayerId("conv8_313_rh")
pts = kernel.transpose().reshape(2, 313, 1, 1)
net.getLayer(class8).blobs = [pts.astype("float32")]
net.getLayer(conv8).blobs = [np.full([1, 313], 2.606, dtype="float32")]
        #-----------------------------------#---------------------#

while capture.isOpened():

    # Capture frames from the camera
    ret, frame = capture.read()
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_frame = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2BGR)
    if ret:
        scaled = frame.astype("float32") / 255.0
        gray_img = cv2.cvtColor(scaled, cv2.COLOR_BGR2GRAY)
        bgr_from_gray = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2BGR)
        
        lab_img = cv2.cvtColor(bgr_from_gray, cv2.COLOR_BGR2LAB)

        #-----------------------------------#---------------------#

       
        # we'll resize the image for the network
        resized = cv2.resize(lab_img, (224, 224))
        # split the L channel
        L = cv2.split(resized)[0]
        # mean subtraction
        L -= 50
        #-----------------------------------#---------------------#

        # predicting the ab channels from the input L channel

        net.setInput(cv2.dnn.blobFromImage(L))
        ab_channel = net.forward()[0, :, :, :].transpose((1, 2, 0))
        # resize the predicted 'ab' volume to the same dimensions as our
        # input image
        ab_channel = cv2.resize(ab_channel, ((lab_img.shape[1], lab_img.shape[0])))


        # Take the L channel from the image
        L = cv2.split(lab_img)[0]
        # Join the L channel with predicted ab channel
        colorized = np.concatenate((L[:, :, np.newaxis], ab_channel), axis=2)

        # Then convert the image from Lab to BGR 
        colorized = cv2.cvtColor(colorized, cv2.COLOR_LAB2BGR)
        colorized = np.clip(colorized, 0, 1)

        # change the image to 0-255 range and convert it from float32 to int
        colorized = (255 * colorized).astype("uint8")

        # Let's resize the images and show them together
        img = cv2.resize(gray_frame,(640,640))
        colorized = cv2.resize(colorized,(640,640))

        result = cv2.hconcat([img,colorized])

        cv2.imshow("Grayscale -> Colour", result)


        # Close the camera if 'q' is pressed
        if cv2.waitKey(1) == ord('q'):
            break

capture.release()
cv2.destroyAllWindows()